In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import  WordNetLemmatizer
from nltk.corpus import stopwords

from gensim.models import Word2Vec

from tqdm import tqdm
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, classification_report

import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
cd /content/drive/MyDrive/Job_Recommendations/

/content/drive/.shortcut-targets-by-id/1gt7f9U5_x7KCF6r7b0N1RkvibfhpTGWe/Job_Recommendations


In [ ]:
import string
from nltk.stem import PorterStemmer

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
cd /content/drive/MyDrive/Job_Recommendations

/content/drive/MyDrive/Job_Recommendations


In [ ]:
users = pd.read_csv('users.csv')
apps = pd.read_csv('apps.csv')
jobs = pd.read_csv('jobs.csv')
work_hist = pd.read_csv('work_history.csv')

In [ ]:
punctuation=list('。，？！：%&~（）、；“”&|,.?!:%&~();""#@【】/-\'$+*`[]{}()')
stop_words = stopwords.words("english")
stop_words.extend(["n't","wo","'m","'s","'ve", "'d", "'ll", "``", "''", "--", "..."])
stop_words.extend(punctuation)
wordnet_lematizer = WordNetLemmatizer()

def pretreatment(comment):
    '''
    remove punctuations, numbers and urls
    lower case conversion
    remove stop words
    lemmatization
    '''

    token_words = word_tokenize(comment)
    token_words = [w.lower() for w in token_words]
    token_words = [w for w in token_words if w not in stop_words]
    token_words =  pos_tag(token_words)
    cleaned_word = []
    for word, tag in token_words:
        if word.isdigit():
            continue
        if tag.startswith('NN'):
            word_lematizer =  wordnet_lematizer.lemmatize(word, pos='n')  # n for noun
        elif tag.startswith('VB'):
            word_lematizer =  wordnet_lematizer.lemmatize(word, pos='v')   # v for verb
        elif tag.startswith('JJ'):
            word_lematizer =  wordnet_lematizer.lemmatize(word, pos='a')   # a for adjective
        elif tag.startswith('R'):
            word_lematizer =  wordnet_lematizer.lemmatize(word, pos='r')   # r for pronoun
        else:
            word_lematizer =  wordnet_lematizer.lemmatize(word)
        cleaned_word.append(word_lematizer)

    return cleaned_word

In [ ]:
jobs.fillna(' ', inplace=True)
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115684 entries, 0 to 115683
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   JobID        115684 non-null  int64 
 1   WindowID     115684 non-null  int64 
 2   Title        115684 non-null  object
 3   City         115684 non-null  object
 4   State        115684 non-null  object
 5   Country      115684 non-null  object
 6   Zip5         115684 non-null  object
 7   StartDate    115684 non-null  object
 8   EndDate      115684 non-null  object
 9   DescCleaned  115684 non-null  object
 10  ReqCleaned   115684 non-null  object
dtypes: int64(2), object(9)
memory usage: 9.7+ MB


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def replace_job_requirements(requirements):
    if str(requirements).startswith("please refer to the job description"):
        return " "
    else:
        return requirements

In [ ]:
jobs['ReqCleaned'] = jobs['ReqCleaned'].apply(lambda x: replace_job_requirements( x))

In [ ]:
na_counts = jobs['DescCleaned'].isna().sum()

In [ ]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115684 entries, 0 to 115683
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   JobID        115684 non-null  int64 
 1   WindowID     115684 non-null  int64 
 2   Title        115684 non-null  object
 3   City         115684 non-null  object
 4   State        115684 non-null  object
 5   Country      115684 non-null  object
 6   Zip5         115684 non-null  object
 7   StartDate    115684 non-null  object
 8   EndDate      115684 non-null  object
 9   DescCleaned  115684 non-null  object
 10  ReqCleaned   115684 non-null  object
dtypes: int64(2), object(9)
memory usage: 9.7+ MB


In [ ]:
na_counts

0

In [ ]:
jobs['combine'] =   jobs['Title'] + ' ' + jobs['DescCleaned'] + ' ' + jobs['ReqCleaned']
list_words = jobs.head(1)['combine'].get(0).split()
print(list_words[7])

full


In [ ]:
jobs['combine'].values

array(['Administrative Assistant opportunity arbor group is a full service independent agency committed to building long term relationship with individuals, businesses and organizations throughout new jersey and pennsylvania in the insurance and employee benefit marketplace. we are looking for an administrative assistant in our bala cynwyd, pa office. essential functions director of first impression by answering incoming calls and greeting all guests in a professional manner. responsible for the administrative support of the arbor owners and team. oversee all aspects of general office coordination including but not limited to repair and maintenance, copying, scanning, faxing, travel arrangements, meeting arrangements, and schedule maintenance. handle the management process for arbor group clients including working with carriers to obtain enforce policies and to maintain up to date files. produce accurate, quality correspondence related to individual clients cases. maintain confidential

In [ ]:
jobs['ReqCleaned'][0]

'required skills/education 3 years work experience. proficient and knowledgeable with microsoft suite including excel, power point, word and outlook. a self starter who is detail oriented and organized. able to set priorities, define goals and manage multi tasked projects good written and verbal communications skills industry experience a plus compensation based on experience   '

In [ ]:
stop_words = set(stopwords.words('english'))
punctuation=list('。，？！：%&~（）、；“”&|,.?!:%&~();""#@【】/-\'$+*`[]{}()')
additional_stop_words = ["n't", "wo", "'m", "'s", "'ve", "'d", "'ll", "``", "''", "--", "...",]
stop_words.update(additional_stop_words)
stop_words.update(punctuation)


def clean_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stemming
    # stemmer = PorterStemmer()
    # tokens = [stemmer.stem(token) for token in tokens]

    return tokens

In [ ]:
from tqdm import tqdm

list_tokens = []
combine = jobs['combine'].values

# print(type(combine))
# jobs['combine'] = jobs['Title'] + jobs['DescCleaned'] + jobs['ReqCleaned']
for row in tqdm(combine):
    # try:
    tokens = clean_text(row)
    #print(tokens)
    list_tokens.append(tokens)
    # except :
    #     print(row)
    #     list_tokens.append(' ')

100%|██████████| 115684/115684 [05:31<00:00, 348.88it/s]


In [ ]:
len(list_tokens)

115684

In [ ]:
jobs.head()

,JobID,WindowID,Title,City,State,Country,Zip5,StartDate,EndDate,DescCleaned,ReqCleaned,combine
0,6,6,Administrative Assistant,Bala Cynwyd,PA,US,19004.0,2012-05-31 17:27:01.63,2012-06-29 23:59:00,opportunity arbor group is a full service inde...,required skills/education 3 years work experie...,Administrative Assistant opportunity arbor gro...
1,177,6,Maintenance Tech,Orlando,FL,US,32808.0,2012-05-13 01:16:58.923,2012-06-12 23:59:59,maintenance tech - hvac certified. for lrg apt...,,Maintenance Tech maintenance tech - hvac certi...
2,179,6,Podium Speaker,Orlando,FL,US,,2012-05-12 09:00:53.553,2012-06-11 23:59:59,vacation village resorts podium speaker are yo...,,Podium Speaker vacation village resorts podium...
3,181,6,Sales Professionals,Orlando,FL,US,,2012-05-12 09:00:55.86,2012-06-11 23:59:59,vacation village resorts seeking sales profess...,,Sales Professionals vacation village resorts s...
4,184,6,ADMINISTRATIVE ASSISTANT,Sanford,FL,US,32773.0,2012-05-14 10:22:27.877,2012-06-13 23:59:59,administrative assistant —— for an executive ...,,ADMINISTRATIVE ASSISTANT administrative assis...


In [ ]:
jobs['Modified'] = list_tokens

In [ ]:
type(list_tokens)

list

In [ ]:
jobs.shape

(115684, 13)

In [ ]:
jobs.head()

,JobID,WindowID,Title,City,State,Country,Zip5,StartDate,EndDate,DescCleaned,ReqCleaned,combine,Modified
0,6,6,Administrative Assistant,Bala Cynwyd,PA,US,19004.0,2012-05-31 17:27:01.63,2012-06-29 23:59:00,opportunity arbor group is a full service inde...,required skills/education 3 years work experie...,Administrative Assistant opportunity arbor gro...,"[administrative, assistant, opportunity, arbor..."
1,177,6,Maintenance Tech,Orlando,FL,US,32808.0,2012-05-13 01:16:58.923,2012-06-12 23:59:59,maintenance tech - hvac certified. for lrg apt...,,Maintenance Tech maintenance tech - hvac certi...,"[maintenance, tech, maintenance, tech, hvac, c..."
2,179,6,Podium Speaker,Orlando,FL,US,,2012-05-12 09:00:53.553,2012-06-11 23:59:59,vacation village resorts podium speaker are yo...,,Podium Speaker vacation village resorts podium...,"[podium, speaker, vacation, village, resorts, ..."
3,181,6,Sales Professionals,Orlando,FL,US,,2012-05-12 09:00:55.86,2012-06-11 23:59:59,vacation village resorts seeking sales profess...,,Sales Professionals vacation village resorts s...,"[sales, professionals, vacation, village, reso..."
4,184,6,ADMINISTRATIVE ASSISTANT,Sanford,FL,US,32773.0,2012-05-14 10:22:27.877,2012-06-13 23:59:59,administrative assistant —— for an executive ...,,ADMINISTRATIVE ASSISTANT administrative assis...,"[administrative, assistant, administrative, as..."


In [ ]:
jobs.to_csv("job_cleaned_change.csv",index=False)

In [ ]:
cd /content/drive/MyDrive/Job_Recommendations

/content/drive/MyDrive/Job_Recommendations


In [ ]:
jobs = pd.read_csv('job_cleaned_change.csv', delimiter='\t',encoding='utf-8')

In [ ]:
jobs.head()

In [ ]:
list_tokens = []

In [ ]:
for row in jobs['combine']:
    if isinstance(row, str) and row.strip():  # Ensure it's a non-empty string
        tokens = row.split()  # Split by whitespace (space, tabs, etc.)
    else:
        tokens = []  # Append an empty list for invalid/empty rows

    # Append tokens to the list
    list_tokens.append(tokens)


In [ ]:
print(list_tokens[1000])

['mtg', 'interim', 'loan', 'documentation', 'specialist', '4', 'home', 'loan', 'processo', 'position', 'short', 'term', 'nature', 'eligible', 'wells', 'fargo', 'benefits', 'job', 'designated', 'use', 'home', 'mortgage', 'periods', 'business', 'conditions', 'demand', 'increased', 'operational', 'capacity', 'responsible', 'performing', 'variety', 'loan', 'documentation', 'duties', 'complex', 'loan', 'packages', 'ensure', 'compliance', 'company', 'policies', 'procedures', 'loan', 'products', 'functions', 'include', 'processing', 'closing', 'compliance', 'loan', 'products', 'interpreting', 'policies', 'analyzing', 'applicant', 'property', 'documentation', 'ordering', 'required', 'verifications', 'documentation', 'subsequent', 'follow-ups', 'may', 'provide', 'guidance', 'training', 'loan', 'documentation', 'team', 'members', 'may', 'manage', 'assigned', 'pipeline', 'loans', 'and/or', 'assign', 'new', 'loans', 'loan', 'team', 'members', 'may', 'act', 'liaison', 'lenders', 'legal', 'departmen

In [ ]:
jobs['Modified'] = list_tokens

In [ ]:
jobs['Modified'][0][0]

"['requir',"

In [ ]:
list_tokens[0][6]

"'profici',"

In [ ]:
jobs['combine'][0].split(' ')[6]

"'profici',"

In [ ]:
w2v_model = Word2Vec.load('./word2vec.model')

FileNotFoundError: [Errno 2] No such file or directory: './word2vec.model'

In [ ]:
jobs.Combine_desc.values

AttributeError: 'DataFrame' object has no attribute 'Combine_desc'

In [ ]:


from gensim.models import word2vec, Word2Vec

def train_word2vec(x):
    '''
    param: x is a list contain all the words
    return: the trained model
    '''

    model = word2vec.Word2Vec(x, vector_size=200, window=2, min_count=2, workers=8,
                             epochs=2, sg=1)
    return model


# 20-30min
w2v_model = train_word2vec(jobs.Modified.values)
w2v_model.save('./word2vec_change.model')



In [ ]:
pwd

'/content/drive/MyDrive/Job_Recommendations'

In [ ]:
w2v_model = Word2Vec.load('./word2vec_change.model')

In [ ]:
my_dict = dict({})
for idx, key in enumerate(w2v_model.wv.key_to_index):
    print(key)
    my_dict[key] = w2v_model.wv[key]

Streaming output truncated to the last 5000 lines.
environmentneeds
appropriateposition
unitsexecellent
experience-ability
recordextensive
funcitiona
activitities
adjustemnt
appropriateupdate
n/aphysical
high-energetic
relationshipmust
odatabases
kirsten.carlson
capellini
profts
goal.lead
activities.formulate
bielke
katie.bielke
oapache
330-253-8160
team/stakeholders
analyst/qa
strategybe
performance.liaison
noit
accountwork
specifications/design/code
hardware/cartridge
discuss/debug
de-activation
05/24/12
952.831.6888
www.ets-uv.com
in.​
paxit
gateway/st
sidebar
extract/analyze
time-boxed
491-9115fax
smooth-running
country.​
www.furniturebrands.com/careers
bienvenidos
basic/less
ato-qca-sr
raisor
b-day
icfmr
egov.com
www.l-3jobs.com
shop.or
rate/rule
patches/upgrades
www.goldmaxusa.com
pay.​
preferredexperience•
facility•
scheduleassist
copays/coinsurance/deductibles
ahfh
jre
grade/band
il-northbrook
31.35
•over
05/29/12
//www.theintuitivegroup-dc.com./
388-2863.
rice/cemli
rn-s
490k


KeyboardInterrupt: 

In [ ]:
len(my_dict)

107078

In [ ]:
len(my_dict)

123714

In [ ]:

w2v_model = Word2Vec.load('./word2vec_change.model')

In [ ]:
work_hist.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,13,6,Test,1,Pennsylvania Mentor
1,13,6,Test,2,Student Worker
2,13,6,Test,3,Internship in Adoption Unit
3,13,6,Test,5,Student Worker - Continuing Education
4,13,6,Test,6,Sales Associate


Perform tfidf vectorization on users work history

In [ ]:
user_id = apps.UserID.unique()
work_history = work_hist[work_hist.UserID.isin(user_id)]
work_history = work_history.drop(columns=["Sequence"]).drop_duplicates()

Pre processing is required for Job titles

In [ ]:
work_history.shape

(89965, 4)

In [ ]:
def sum_with_space(series):
    return ' '.join(series)

work_history_tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, max_features=50, stop_words='english')
work_history_tf_matrix = work_history_tf.fit_transform(work_hist.groupby('UserID')['JobTitle'].apply(sum_with_space))

In [ ]:
class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        '''
        param: sentences: the list of corpus
               sen_len: the max length of each sentence
               w2v_path: the path storing word emnbedding model
        '''

        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []

    def get_w2v_model(self):
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
      ## Adding a new word to the word to vec with a tensor of uniform distribution
    def add_embedding(self, word):
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)

    def make_embedding(self, load=True):
        print("Get embedding ...")
        if load:
            print("loading word2vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        for i, word in enumerate(self.embedding.wv.vocab):
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        self.add_embedding("")
        self.add_embedding("")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix

    def pad_sentence(self, sentence):
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx[''])
        assert len(sentence) == self.sen_len
        return sentence

    def sentence_word2idx(self):
        '''
        change words in sentences into idx in embedding_matrix
        '''
        ## Given a sentence find the ids of words in sentence. If any unknown word replace with empty spaces
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx[''])
            sentence_idx = self.pad_sentence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)

    def labels_to_tensor(self, y):
        return torch.LongTensor(y)

Can apply CNN LSTM

In [ ]:
## Model



class TextCNN(nn.Module):
    def __init__(self, channels, kernel_size, pool_size, dim, method='max'):
        super(TextCNN, self).__init__()
        self.net1 = nn.Sequential(
            nn.Conv2d(1, channels, kernel_size[0]),
            nn.BatchNorm2d(channels),
            nn.ReLU(),
            nn.MaxPool2d(pool_size)
        )
        self.net2 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size[1]),
            nn.BatchNorm2d(channels),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1, dim))
        )
        if method == 'max':
            self.pool = nn.AdaptiveMaxPool2d((1, dim))
        elif method == 'mean':
            self.pool = nn.AdaptiveAvgPool2d((1, dim))
        else:
            raise ValueError('method {} not exist'.format(method))

    def forward(self, x):
        x = self.net1(x)
        x = self.net2(x).squeeze(2)
        x = self.pool(x).squeeze(1)
        return x


class MLP(nn.Module):
    def __init__(self, input_size, output_size, dropout):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(input_size, input_size),
            nn.ReLU(),
            nn.Linear(input_size, output_size),
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.net(x)
        return x


class PJFNN(nn.Module):
    def __init__(self, embedding, input_dim, channels=1, dropout=0.5, fix_embedding=True):
        super(PJFNN, self).__init__()
        self.dim = embedding.size(1)
        self.user_dim = input_dim
        self.channels = channels
        self.embedding = nn.Embedding(embedding.size(0), embedding.size(1))
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False if fix_embedding else True
        # self.emb = nn.Embedding.from_pretrained(
        #    torch.from_numpy(np.load( os.path.join(args['dataset']['path'], 'emb.npy') )),
        #    freeze=False,
        #    padding_idx=0
        #)

        # self.geek_layer = TextCNN(
        #     channels=args['dataset']['max_sent_num']['geek'],
        #     kernel_size=[(5, 1), (3, 1)],
        #     pool_size=(2, 1),
        #     dim=dim,
        #     method='max'
        # )
        self.user_layer = MLP(self.user_dim , 64, dropout=dropout)
        self.linear_transform = nn.Linear(200, 64)
        self.job_layer = TextCNN(
            channels=self.channels,
            kernel_size=[(3, 1), (3, 1)],
            pool_size=(2, 1),
            dim=200,
            method='mean'
        )

        self.mlp = MLP(
            input_size=128,
            output_size=1,
            dropout=dropout
        )


    def forward(self, job, user):
        job = self.embedding(job)
        job = job.unsqueeze(1)
        job = self.job_layer(job)
        user = self.user_layer(user)
        job = self.linear_transform(job)
        x = torch.cat((user,job),dim=1)
        x = self.mlp(x).squeeze(1)
        return x



In [ ]:
class JobUserDataset(data.Dataset):
    '''
    Expected data shape like:(data_num, data_len)
    '''
    def __init__(self, job, user, label):
        self.job = job
        self.user = user
        self.label = label

    def __getitem__(self, idx):
        if self.label is None: return self.job[idx], self.user[idx]
        return self.job[idx], self.user[idx], self.label[idx]

    def __len__(self):
        return len(self.job)

In [ ]:
jobs.head()

,JobID,WindowID,Title,City,State,Country,Zip5,StartDate,EndDate,DescCleaned,ReqCleaned,combine,Modified
0,6,6,Administrative Assistant,Bala Cynwyd,PA,US,19004.0,2012-05-31 17:27:01.63,2012-06-29 23:59:00,opportunity arbor group is a full service inde...,required skills/education 3 years work experie...,Administrative Assistant opportunity arbor gro...,"[administrative, assistant, opportunity, arbor..."
1,177,6,Maintenance Tech,Orlando,FL,US,32808.0,2012-05-13 01:16:58.923,2012-06-12 23:59:59,maintenance tech - hvac certified. for lrg apt...,,Maintenance Tech maintenance tech - hvac certi...,"[maintenance, tech, maintenance, tech, hvac, c..."
2,179,6,Podium Speaker,Orlando,FL,US,,2012-05-12 09:00:53.553,2012-06-11 23:59:59,vacation village resorts podium speaker are yo...,,Podium Speaker vacation village resorts podium...,"[podium, speaker, vacation, village, resorts, ..."
3,181,6,Sales Professionals,Orlando,FL,US,,2012-05-12 09:00:55.86,2012-06-11 23:59:59,vacation village resorts seeking sales profess...,,Sales Professionals vacation village resorts s...,"[sales, professionals, vacation, village, reso..."
4,184,6,ADMINISTRATIVE ASSISTANT,Sanford,FL,US,32773.0,2012-05-14 10:22:27.877,2012-06-13 23:59:59,administrative assistant —— for an executive ...,,ADMINISTRATIVE ASSISTANT administrative assis...,"[administrative, assistant, administrative, as..."


In [ ]:
## Cleaning users data

users.replace({"CurrentlyEmployed":{"Yes":1,"No":0}}, inplace=True)
users.replace({"ManagedOthers":{"Yes":1,"No":0}}, inplace=True)
users.replace({"DegreeType":{"None":0,"High School":1, "Vocational":2, "Associate's":3, "Bachelor's":4, "Master's":5, "PhD":6}},
                 inplace=True)

In [ ]:
user_city_dict = dict(zip(users['UserID'], users['City']))
user_state_dict = dict(zip(users['UserID'], users['State']))

jobs_city_dict = dict(zip(jobs['JobID'], jobs['City']))
jobs_state_dict = dict(zip(jobs['JobID'], jobs['State']))

In [ ]:
city = []
state = []
for index, row in tqdm(apps.iterrows(), total=len(apps)):
    city.append(1 if jobs_city_dict[row['JobID']] == user_city_dict[row['UserID']] else 0)
    state.append(1 if jobs_state_dict[row['JobID']] == user_state_dict[row['UserID']] else 0)

apps["City"] = city
apps["State"] = state

100%|██████████| 207314/207314 [00:10<00:00, 19455.35it/s]


In [ ]:
apps.head()

,UserID,JobID,label,City,State
0,13,821691,1,0,1
1,13,484404,0,0,0
2,64,666073,1,1,1
3,64,281940,1,1,1
4,64,337025,1,1,1


In [ ]:

train_user = users[users.Split=="Train"].UserID.values
test_user = users[users.Split=="Test"].UserID.values
train_data = apps[apps.UserID.isin(train_user)]
test_data = apps[apps.UserID.isin(test_user)]
#groups = train_data.groupby("UserID")

In [ ]:
groups = train_data.groupby("UserID")
job_train = []
user_train = np.zeros((1, 58))  # Initialize an empty array for user features
Y_train = []

for u_id, group in tqdm(groups):
    # Extract user features
    user = users[users.UserID == u_id][["DegreeType", "WorkHistoryCount", "TotalYearsExperience",
                                        "CurrentlyEmployed", "ManagedOthers", "ManagedHowMany"]]
    u_idx = user.index.values[0]
    temp_user_feature = np.concatenate((user.values, work_history_tf_matrix[u_idx, :].toarray()), axis=1)

    # Extend job_test with job descriptions
    job_id_values = group.JobID.values
    jobs_filter = jobs[jobs.JobID.isin(job_id_values)]
    job_train.extend(jobs_filter['Modified'].values.tolist())

    # Create user features for each job in the group
    user_feature = np.tile(temp_user_feature, (len(job_id_values), 1))  # Repeat user features for each job
    user_feature = np.concatenate((user_feature, group[["State", "City"]].values), axis=1)  # Concatenate State and City

    # Append user features to user_test
    user_train = np.concatenate((user_train, user_feature), axis=0)

    # Extend Y_test with labels
    Y_train.extend(group.label.values.tolist())

# Remove the initial zeros array used for initialization
user_train = user_train[1:]


100%|██████████| 21581/21581 [07:42<00:00, 46.63it/s]


In [68]:
np.save('user_train_change.npy', user_train)
np.save('Y_train.npy_change', Y_train)
np.save('job_train_change.npy',job_train)

In [ ]:
groups = test_data.groupby("UserID")
job_test = []
user_test = np.zeros((1, 58))  # Initialize an empty array for user features
Y_test = []

for u_id, group in tqdm(groups):
    # Extract user features
    user = users[users.UserID == u_id][["DegreeType", "WorkHistoryCount", "TotalYearsExperience",
                                        "CurrentlyEmployed", "ManagedOthers", "ManagedHowMany"]]
    u_idx = user.index.values[0]
    temp_user_feature = np.concatenate((user.values, work_history_tf_matrix[u_idx, :].toarray()), axis=1)

    # Extend job_test with job descriptions
    job_id_values = group.JobID.values
    jobs_filter = jobs[jobs.JobID.isin(job_id_values)]
    job_test.extend(jobs_filter['Modified'].values.tolist())

    # Create user features for each job in the group
    user_feature = np.tile(temp_user_feature, (len(job_id_values), 1))  # Repeat user features for each job
    user_feature = np.concatenate((user_feature, group[["State", "City"]].values), axis=1)  # Concatenate State and City

    # Append user features to user_test
    user_test = np.concatenate((user_test, user_feature), axis=0)

    # Extend Y_test with labels
    Y_test.extend(group.label.values.tolist())

# Remove the initial zeros array used for initialization
user_test = user_test[1:]


100%|██████████| 552/552 [00:02<00:00, 186.21it/s]


In [ ]:
np.save('user_test_change.npy', user_test)
np.save('Y_test_change.npy', Y_test)
np.save('job_test_change.npy',job_test)

In [ ]:
Y_train = Y_train[:]

In [ ]:
class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        '''
        param: sentences: the list of corpus
               sen_len: the max length of each sentence
               w2v_path: the path storing word emnbedding model
        '''

        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []

    def get_w2v_model(self):
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size

    def add_embedding(self, word):
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)

    def make_embedding(self, load=True):
        print("Get embedding ...")
        if load:
            print("loading word2vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        for i, word in enumerate(self.embedding.wv.key_to_index.keys()):
            #print(word)
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding.wv[word])
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix

    def pad_sentence(self, sentence):
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx['<PAD>'])
        assert len(sentence) == self.sen_len
        return sentence

    def sentence_word2idx(self):
        '''
        change words in sentences into idx in embedding_matrix
        '''
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx['<UNK>'])
            sentence_idx = self.pad_sentence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)

    def labels_to_tensor(self, y):
        return torch.LongTensor(y)

In [ ]:
len(job_train)

191220

In [ ]:
train_len = len(job_train)
job_train.extend(job_test)
Y_train.extend(Y_test)
sen_len = 200
preprocess = Preprocess(job_train, sen_len, w2v_path="word2vec_change.model")
embedding = preprocess.make_embedding(load=True)
x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(Y_train)
torch.save(embedding,"./embedding.pt")

Get embedding ...
loading word2vec model ...


<ipython-input-42-8ef69cb83719>:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  self.embedding_matrix = torch.tensor(self.embedding_matrix)


total words: 123716


In [ ]:
train_len

191221

In [ ]:
train_nos = 150000
val_nos = 20000

train_x = x[:train_nos]
train_y = y[:train_nos]
val_x = x[train_nos+1:train_nos+val_nos]
val_y = y[train_nos+1:train_nos+val_nos]
test_x = x[train_nos+val_nos+1:]
test_y = y[train_nos+val_nos+1:]
train_user = torch.from_numpy(user_train[:train_nos])
val_user = torch.from_numpy(user_train[train_nos+1:train_nos+val_nos])
test_user = torch.from_numpy(user_test[train_nos+val_nos+1:])
train_dataset = JobUserDataset(train_x, train_user, train_y)
val_dataset = JobUserDataset(val_x, val_user, val_y)
test_dataset = JobUserDataset(test_x, test_user, test_y)
# torch.save(train_dataset,"train.dataset")
# torch.save(val_dataset,"val.dataset")
# torch.save(test_dataset, "test.dataset")


# train_dataset = torch.load("train.dataset")
# val_dataset = torch.load("val.dataset")
# test_dataset = torch.load("test.dataset")
# embedding = torch.load("embedding.pt")
batch_size = 5
train_loader = DataLoader(dataset= train_dataset, batch_size = batch_size, shuffle = False)
val_loader = DataLoader(dataset = val_dataset, batch_size = batch_size, shuffle = False)
test_loader =DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)



In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA (GPU) is available")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) is not available, falling back to CPU")

CUDA (GPU) is available


In [ ]:
device = 'cpu'

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, channels, kernel_size, pool_size, dim, method='max'):
        super(TextCNN, self).__init__()
        self.net1 = nn.Sequential(
            nn.Conv2d(1, channels, kernel_size[0]),
            nn.BatchNorm2d(channels),
            nn.ReLU(),
            nn.MaxPool2d(pool_size)
        )
        self.net2 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size[1]),
            nn.BatchNorm2d(channels),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1, dim))
        )
        if method == 'max':
            self.pool = nn.AdaptiveMaxPool2d((1, dim))
        elif method == 'mean':
            self.pool = nn.AdaptiveAvgPool2d((1, dim))
        else:
            raise ValueError('method {} not exist'.format(method))

    def forward(self, x):
        x = self.net1(x)
        x = self.net2(x).squeeze(2)
        x = self.pool(x).squeeze(1)
        return x

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, dropout):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(input_size, input_size),
            nn.ReLU(),
            nn.Linear(input_size, output_size),
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.net(x)
        return x

In [ ]:
class PJFNN(nn.Module):
    def __init__(self, embedding, input_dim, channels=1, dropout=0.5, fix_embedding=True):
        super(PJFNN, self).__init__()
        self.dim = embedding.size(1)
        self.user_dim = input_dim
        self.channels = channels
        self.embedding = nn.Embedding(embedding.size(0), embedding.size(1))
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False if fix_embedding else True
        # self.emb = nn.Embedding.from_pretrained(
        #    torch.from_numpy(np.load( os.path.join(args['dataset']['path'], 'emb.npy') )),
        #    freeze=False,
        #    padding_idx=0
        #)

        # self.geek_layer = TextCNN(
        #     channels=args['dataset']['max_sent_num']['geek'],
        #     kernel_size=[(5, 1), (3, 1)],
        #     pool_size=(2, 1),
        #     dim=dim,
        #     method='max'
        # )
        self.user_layer = MLP(self.user_dim , 64, dropout=dropout)
        self.linear_transform = nn.Linear(200, 64)
        self.job_layer = TextCNN(
            channels=self.channels,
            kernel_size=[(5, 1), (5, 1)],
            pool_size=(2, 1),
            dim=200,
            method='mean'
        )

        self.mlp = MLP(
            input_size=128,
            output_size=1,
            dropout=dropout
        )


    def forward(self, job, user):
        job = self.embedding(job)
        job = job.unsqueeze(1)
        job = self.job_layer(job)
        user = self.user_layer(user)
        job = self.linear_transform(job)
        x = torch.cat((user,job),dim=1)
        x = self.mlp(x).squeeze(1)
        return x

In [ ]:
class PJFNN_LSTM(nn.Module):
    def __init__(self, embedding, input_dim, hidden_dim=256, num_layers=1, dropout=0., fix_embedding=True):
        super(PJFNN_LSTM, self).__init__()
        self.embedding_dim = embedding.size(1)
        self.user_dim = input_dim
        self.embedding = nn.Embedding(embedding.size(0), embedding.size(1))
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False if fix_embedding else True
        # self.emb = nn.Embedding.from_pretrained(
        #    torch.from_numpy(np.load( os.path.join(args['dataset']['path'], 'emb.npy') )),
        #    freeze=False,
        #    padding_idx=0
        #)

        # self.geek_layer = TextCNN(
        #     channels=args['dataset']['max_sent_num']['geek'],
        #     kernel_size=[(5, 1), (3, 1)],
        #     pool_size=(2, 1),
        #     dim=dim,
        #     method='max'
        # )
        self.user_layer = MLP(self.user_dim , 64, dropout=dropout)
        self.linear_transform = nn.Linear(256, 64)
        self.job_layer = nn.LSTM(self.embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)

        self.classifier = nn.Sequential(nn.Dropout(dropout), nn.Linear(128, 1), nn.Sigmoid())


    def forward(self, job, user):
        job = self.embedding(job)
        job, _ = self.job_layer(job, None)
        job = job[:,-1,:]
        user = self.user_layer(user)
        job = self.linear_transform(job)
        x = torch.cat((user,job),dim=1)
        x = self.classifier(x).squeeze(1)
        return x

In [ ]:
x[9]

tensor([ 43146,     95,     82,    267,     13,    493,    177,    431,    290,
           176,     95,     82,    267,    167,  26249,  43146,     11,  12155,
          1286,     13,   1031,     44,   1409,    274,    794,   3404,     95,
           101,   2663,    847,   6246,     73,   5506,    847,     82,    535,
          1952,   1683,   1244,     70,    485,   4091,   1577,     28,   3590,
          1081,   3503,    713,   1278,    721,    432,    254,   3404,    133,
           189,    163,    721,    921,    391,   1278,    272,     75,     70,
          1070,   1086,   1344,     84,     27,    425,    288,    534,    880,
           170,     68,    300,    827,  12732,   2009,   4010,   1067,     56,
          2213,   1194,    341,    159,   3104,    809,    557,   1855,     80,
           171,   1227,      2,   1847,   1067,    573,    827,   1339,   3754,
         11800,   3405,    674,    557,     75,   1533,   9315,     28,   2277,
           288,   7935,     10,      3, 

In [ ]:


def training(batch_size, n_epoch, lr, train, valid, model, device, model_name, model_dir="./"):
    # summary model parameters
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print("\nstart training, total parameter:{}, trainable:{}\n".format(total, trainable))
    model.to(device)
    model.train()
    criterion = nn.BCELoss()
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    total_loss, total_acc, best_acc = 0, 0, 0
    train_losses, val_losses = [], []
    train_acc, val_acc = [], []
    pred_label = []
    y_label = []

    for epoch in range(n_epoch):
        start_time = time.time()
        total_loss, total_acc = 0, 0
        # training
        for i, (jobs, users, labels) in enumerate(train):
            jobs = jobs.to(device)
            users = users.to(torch.float32)
            users = users.to(device)
            labels = labels.to(device)
            labels = labels.to(torch.float32)
            model.zero_grad()
            outputs = model(jobs, users)
            if torch.isnan(outputs).any():
                  print('True')
                  loss = 0.0
            else:
                  loss = criterion(outputs, labels)
            loss_tensor = torch.tensor(loss, requires_grad=True)

            loss_tensor.backward()
            optimizer.step()
            total_loss += loss_tensor.item()
            pred_label.extend([0 if i<0.5 else 1 for i in list(outputs.cpu().detach().numpy())])
            y_label.extend(list(labels.cpu().detach().numpy()))
        train_losses.append(total_loss/t_batch)
        train_acc.append(accuracy_score(y_label, pred_label))
        print('[ Epoch{}: {}/{}] '.format(
                epoch+1, i+1, t_batch))
        print('\nTrain | Loss:{:.5f} Time:{:.6f}'.format(total_loss/t_batch, time.time()-start_time))

        # evaluation
        model.eval()
        with torch.no_grad():
            pred_label = []
            y_label = []
            total_loss, total_acc = 0, 0
            for i, (jobs, users, labels) in enumerate(valid):
                jobs = jobs.to(device)
                users = users.to(torch.float32)
                users = users.to(device)
                labels = labels.to(torch.float32)
                labels = labels.to(device)
                outputs = model(jobs, users)
                if torch.isnan(outputs).any():
                  print('true')
                  loss = 0.0
                else:
                    loss = criterion(outputs, labels)
                loss_tensor = torch.tensor(loss, requires_grad=True)
                total_loss += loss.item()
                pred_label.extend([0 if i<0.5 else 1 for i in list(outputs.cpu().detach().numpy())])
                y_label.extend(list(labels.cpu().detach().numpy()))
            # print('\nVal | Loss:{:.5f} Time:{:.6f}'.format(total_loss/v_batch, time.time()-start_time))
            val_losses.append(total_loss/v_batch)
            total_acc =  accuracy_score(y_label, pred_label)
            val_acc.append(total_acc)
            print('\nVal | ACC:{:.5f} Time:{:.6f}'.format(total_acc, time.time()-start_time))
            if total_acc > best_acc:
                best_acc = total_acc
                torch.save(model, "{}/{}_ckpt.model".format(model_dir, model_name))
                print('save model with acc {:.3f}'.format(total_acc))
        print('------------------------------------------------------')
        model.train()
    return train_losses, val_losses, train_acc, val_acc


def test(model, loader):
    predictions = []
    y_labels = []
    pred = []
    model.eval()
    for i, (jobs, users, labels) in enumerate(loader):
        jobs = jobs.to(device)
        users = users.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.float32)
        outputs = model(jobs, users)
        if torch.isnan(outputs).any():
               print('True')
        pred.extend(list(outputs.cpu().detach().numpy()))
        predictions.extend([0 if i<0.5 else 1 for i in list(outputs.cpu().detach().numpy())])
        y_labels.extend(list(labels.cpu().detach().numpy()))
        report = classification_report(y_labels,predictions,digits=4)
    report = report.splitlines()
    columns = ['class'] + report[0].split()
    col_1, col_2, col_3, col_4, col_5 = [], [], [], [], []
    for row in report[1:]:
        if len(row.split()) != 0:
            row = row.split()
            if len(row) < 5:
                col_1.append(row[0])
                col_2.append('')
                col_3.append('')
                col_4.append(row[1])
                col_5.append(row[2])
            elif len(row) > 5:
                col_1.append(row[0] + ' ' + row[1])
                col_2.append(row[2])
                col_3.append(row[3])
                col_4.append(row[4])
                col_5.append(row[5])
            else:
                col_1.append(row[0])
                col_2.append(row[1])
                col_3.append(row[2])
                col_4.append(row[3])
                col_5.append(row[4])
    result = pd.DataFrame()
    col_1.append("overall")
    col_2.append(precision_score(y_labels, predictions))
    col_3.append(recall_score(y_labels, predictions))
    col_4.append(f1_score(y_labels, predictions))
    col_5.append(roc_auc_score(y_labels, pred))
    result[columns[0]] = col_1
    result[columns[1]] = col_2
    result[columns[2]] = col_3
    result[columns[3]] = col_4
    result[columns[4]] = col_5
    print("——————Test——————")
#     print(result)
    return result



In [ ]:
def main():
    fix_embedding = False
    input_dim = train_dataset[0][1].shape[0]  # You need to define `train_dataset` or replace it with appropriate code
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the appropriate device
    #model = PJFNN_LSTM(embedding, input_dim, dropout=0.7, channels=32, fix_embedding=fix_embedding).to(device)
    model = PJFNN_LSTM(embedding, input_dim, fix_embedding=fix_embedding).to(device)

    epoch = 10
    lr = 0.0005
    model_dir = './'

    # Ensure train_loader and val_loader are also moved to the device
    train_losses, val_losses, train_acc, val_acc = training(batch_size, epoch, lr, train_loader, val_loader, model, device, "textCNN", model_dir)

    # Assuming you have saved the model using `torch.save(model.state_dict(), "textCNN_ckpt.pth")`
    # Load the model's state_dict and move it to the appropriate device for testing
    model.load_state_dict(torch.load("textCNN_ckpt.pth", map_location=device))

    test(model, test_loader)  # Ensure `test` function receives the model on the correct device

if __name__ == "__main__":
    #torch.backends.cudnn.benchmark = False
    #os.environ["TORCH_USE_CUDA_DSA"] = "1"
    main()


In [68]:


fix_embedding = False
input_dim = train_dataset[0][1].shape[0]
model = PJFNN(embedding, input_dim, dropout=0.7, channels=32, fix_embedding=fix_embedding)
# model


epoch = 10
lr =  0.0005
model_dir = './'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_losses, val_losses, train_acc, val_acc = training(batch_size, epoch, lr, train_loader, val_loader, model, device, "textCNN",  model_dir )


test(torch.load("textCNN_ckpt.model"), test_loader)

